# Creación csv maestro regional
Contiene datos de todas las fuentes por region en formato csv

## Importar librerias

In [1]:
import pandas as pd
import os

## Funcion para extraer el código de region
Funcion que permite extraer el código de region, desde el codigo de comuna.

In [2]:
# funcion para extraer el código de región
def extraer_cod_region(codigo):
    codigo = codigo.strip() 
    if len(codigo) == 5:
        return codigo[:2]   # Primeros 2 dígitos
    elif len(codigo) == 4:
        return codigo[:1]   # Primer dígito
    else:
        return None

## Cargar rutas estadisticas Cesnso

In [3]:
ruta_data = "../../data/processed_data/"
ruta_estadisticas = "../../data/processed_data/preliminar_estadisticas/pais/estadisticas_pais.csv"
df = pd.read_csv(ruta_estadisticas)

Eliminamos la fila que contiene el total pais y cambiamos nombres a columnas.

In [4]:
df = df[df['Código región'] != 0]
df = df.rename(columns={
    'Código región': 'cod_region',
    'poblacion_total': 'total_poblacion',
    'poblacion_migrante': 'total_poblacion_migrante',
    'porcentaje': 'porcentaje_migrante_total'
})
df['cod_region'] = df['cod_region'].astype(str)

## Cargar datos fonasa
Se deben obtener los datos procesados de fonasa, estos vienen con informacion detallada por comuna por lo que debemos agruparla por region, para posteriormente recalcular los valores totales y porcentuales.

In [5]:
# FONASA
df_fonasa = pd.read_csv(os.path.join(ruta_data, "fonasa.csv"))
df_fonasa['Código comuna'] = df_fonasa['Código comuna'].astype(str)

# Extraer codigo de región
df_fonasa['cod_region'] = df_fonasa['Código comuna'].apply(extraer_cod_region)

# Agrupamos por Región y Código de Región
df_fonasa = df_fonasa.groupby(['cod_region', 'REGIÓN'])[[
    'BENEFICIARIOS_CHILENOS',
    'BENEFICIARIOS_EXTRANJEROS'
]].sum().reset_index()
df_fonasa['cod_region'] = df_fonasa['cod_region'].astype(str)

# Calculo del total de beneficiarios y porcentaje de migrantes en fonasa
df_fonasa['TOTAL_BENEFICIARIOS'] = df_fonasa['BENEFICIARIOS_CHILENOS'] + df_fonasa['BENEFICIARIOS_EXTRANJEROS']
df_fonasa['porcentaje_migrante_fonasa'] = (df_fonasa['BENEFICIARIOS_EXTRANJEROS'] / df_fonasa['TOTAL_BENEFICIARIOS']) * 100

## Cargar datos Casen
Se deben cargar los datos ya procesados de casen, en este caso se encuentra ya agrupado por región.

In [6]:
# CASEN
df_casen = pd.read_csv(os.path.join(ruta_data, "casen_niveles_regiones.csv"))
df_casen['region'] = df_casen['region'].astype(str)
df_casen = df_casen.rename(columns={"region": "cod_region"})

## Cargar datos red hospitalaria
Se deben obtener los datos procesados de red hospitalaria publica, estos vienen con informacion detallada por comuna por lo que debemos agruparla por region.

In [7]:
df_red = pd.read_csv(os.path.join(ruta_data, "red_hospitalaria_publica.csv"))
df_red = df_red.rename(columns={"Código Comuna": "Código comuna"})
df_red['Código comuna'] = df_red['Código comuna'].astype(str)

# Extraer codigo de región
df_red['cod_region'] = df_red['Código comuna'].apply(extraer_cod_region)

#  Agrupamos por Región y Código de Región, y sumamos
df_red = df_red.groupby(['cod_region'])[[
    'Total_centros'
]].sum().reset_index()

## Cargar datos de atenciones publicas
Se deben cargar los datos ya procesados del total de atenciones, en este caso se encuentra ya agrupado por región.

In [8]:
# ATENCIONES
df_atenc = pd.read_csv(os.path.join(ruta_data, "total_atenciones_promedio_regiones.csv"))
df_atenc = df_atenc.rename(columns={"IdRegion": "cod_region"})
df_atenc['cod_region'] = df_atenc['cod_region'].astype(str)

## Cargar datos área comunal
Se deben obtener los datos procesados de area segun comuna, debemos agruparla por region.

In [9]:
# ÁREA
df_area = pd.read_csv(os.path.join(ruta_data, "Area_territorial.csv"))
df_area['Código comuna'] = df_area['Código comuna'].astype(str)

# Extraer codigo de región
df_area['cod_region'] = df_area['Código comuna'].apply(extraer_cod_region)

# Agrupamos por Región y Código de Región, y sumamos
df_area = df_area.groupby(['cod_region'])[[
    'Area'
]].sum().reset_index()

## Union de los csv
Se unen todos los csv segun su codigo regional

In [10]:
# Hacemos Left Join para no perder comunas aunque no tengan datos en otras tablas
df_final = df.merge(df_fonasa[['cod_region', 'porcentaje_migrante_fonasa']], on='cod_region', how='left')
df_final = df_final.merge(df_casen[['cod_region', 'ruralidad', 'nivel_promedio', 'desviacion_estandar']], on='cod_region', how='left')
df_final = df_final.merge(df_red[['cod_region', 'Total_centros']], on='cod_region', how='left')
df_final = df_final.merge(df_atenc[['cod_region', 'promedio_pacientes', 'porcentaje_migrantes_promedio']], on='cod_region', how='left')
df_final = df_final.merge(df_area[['cod_region', 'Area']], on='cod_region', how='left')

# Rellenar nulos lógicos (ej: si no cruza con centros, es que tiene 0 centros)
df_final['Total_centros'] = df_final['Total_centros'].fillna(0)

## Calculo de Tasas

In [11]:
# Tasa de Centros por 10.000 habitantes
df_final['tasa_centros_10k'] = (df_final['Total_centros'] / df_final['total_poblacion']) * 10000

# Tasa de Atenciones por 10.000 habitantes
df_final['tasa_atenciones_10k'] = (df_final['promedio_pacientes'] / df_final['total_poblacion']) * 10000

# Densidad Poblacional (Hab/km2)
df_final['densidad_poblacional'] = df_final['total_poblacion'] / df_final['Area']

# redondear
cols_round = ['tasa_centros_10k', 'tasa_atenciones_10k', 'densidad_poblacional']
df_final[cols_round] = df_final[cols_round].round(2)

In [12]:
df_final.head()

,cod_region,Región,total_poblacion,total_poblacion_migrante,porcentaje_migrante_total,porcentaje_migrante_fonasa,ruralidad,nivel_promedio,desviacion_estandar,Total_centros,promedio_pacientes,porcentaje_migrantes_promedio,Area,tasa_centros_10k,tasa_atenciones_10k,densidad_poblacional
0,15,Arica y Parinacota,244569.0,36188.0,14.80,19.696832,0.100466,1.888003,0.896013,21,13714.500000,11.35,16873.0,0.86,560.76,14.49
1,1,Tarapacá,369806.0,84935.0,22.97,26.727673,0.077091,2.738465,1.967768,45,27774.583333,18.12,42226.0,1.22,751.06,8.76
2,2,Antofagasta,635416.0,124006.0,19.52,25.484516,0.090597,2.264260,1.181571,50,36004.750000,18.13,126049.0,0.79,566.63,5.04
3,3,Atacama,299180.0,25167.0,8.41,10.521071,0.118078,2.697148,1.889598,48,23345.083333,3.43,75177.0,1.60,780.30,3.98
4,4,Coquimbo,832864.0,44578.0,5.35,5.929361,0.237980,2.872632,1.834501,167,59931.500000,2.51,40580.0,2.01,719.58,20.52


## Se crea csv maestro regiones

In [13]:
salida = os.path.join(ruta_data, "maestro/csv_maestro_regiones.csv")
df_final.to_csv(salida, index=False, encoding='utf-8')